<h3> Perturbation theory to approximate and infinite square well with a linear term V(x) = alpha x </h3>

In [ ]:
import math
import scipy.constants as sc


"""
Calculating the energy for any n without an added perturbation
n       = number of excited state
mass    = mass of particle in the well
length  = length of the well
"""
def unperturbed_energy(n, mass, length):
    numerator = math.pow(n,2) * math.pow(math.pi, 2) * math.pow(sc.hbar, 2)
    denominator = 2 * mass * math.pow(length, 2)

    return numerator / denominator

"""
Calculating the first order correction.
Found by solving ⟨1|V1|1⟩
This correction is generalized for all n
"""
def first_order_correction(alpha, length):
    numerator = alpha * length

    return numerator / 2


"""
Calculating the second order correction for the ground state.
Found by solving:
        a^2 SUM(m=/=n) |⟨n|V1|n⟩|^2 / E(0)n - E(0)m
"""
def second_order_correction_groundstate(alpha, mass, length, max_n=10):
    # Split into left-hand-side (LHS) and right-hand-side(RHS) which is the sum over even n
    numerator = 128 * mass * math.pow(length, 4)
    denominator = math.pow(math.pi, 4) * math.pow(sc.hbar,2)
    LHS = numerator / denominator

    # Now summing over even n
    sum_range = range(2,max_n, 2)
    sum_result = 0
    for i in sum_range:
        temp = math.pow(i,2) / (math.pow(i-1, 5)*math.pow(i+1,5))
        sum_result+=temp

    
    return LHS * sum_result

""" 
Combining the unperturbed energy with the 2 corrections for our final result
"""
def energy_groundstate_second_order_perturbation(e01, e11, e21):
    return e01 + e11 + e21
    

In [6]:
particle_mass = sc.electron_mass # using an electron
well_length = 1e-9

ground_state = unperturbed_energy(1,particle_mass,well_length)
print(ground_state)

6.024667394036591e-20
